#### Challenge 42: Bleichenbacher's e=3 RSA Attack

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)

In [1]:
from Crypto.Util import number
from Crypto.Random import random
from Crypto.Hash.SHA256 import SHA256Hash

import cryptopals as cp
import sha1

<div class="alert alert-block alert-info">

<div class="alert alert-block alert-warning">    

#### **Crypto-tourism informational placard.**

This attack broke Firefox's TLS certificate validation several years ago. You could write a Python script to fake an RSA signature for any certificate. We find new instances of it every other year or so.

</div>
    
RSA with an encrypting exponent of 3 is popular, because it makes the RSA math faster.

With `e=3` RSA, encryption is just cubing a number mod the public encryption modulus:

`c = m ** 3 % n`

    
`e=3` is secure as long as we can make assumptions about the message blocks we're encrypting. The worry with low-exponent RSA is that the message blocks we process won't be large enough to wrap the modulus after being cubed. The block `00:02` (imagine sufficient zero-padding) can be "encrypted" in `e=3` RSA; it is simply `00:08`.

When RSA is used to sign, rather than encrypt, the operations are reversed; the verifier "decrypts" the message by cubing it. This produces a "plaintext" which the verifier checks for validity.

When you use RSA to sign a message, you supply it a block input that contains a message digest. The PKCS1.5 standard formats that block as:

`00h 01h ffh ffh ... ffh ffh 00h ASN.1 GOOP HASH`

As intended, the `ffh` bytes in that block expand to fill the whole block, producing a "right-justified" hash (the last byte of the hash is the last byte of the message).

There was, 7 years ago, a common implementation flaw with RSA verifiers: they'd verify signatures by "decrypting" them (cubing them modulo the public exponent) and then "parsing" them by looking for `00h 01h ... ffh 00h ASN.1 HASH`.

This is a bug because it implies the verifier isn't checking all the padding. If you don't check the padding, you leave open the possibility that instead of hundreds of ffh bytes, you have only a few, which if you think about it means there could be squizzilions of possible numbers that could produce a valid-looking signature.

How to find such a block? Find a number that when cubed     
- (a) doesn't wrap the modulus (thus bypassing the key entirely) and   
- (b) produces a block that starts `"00h 01h ffh ... 00h ASN.1 HASH"`.

There are two ways to approach this problem:

- You can work from Hal Finney's writeup, available on Google, of how Bleichenbacher explained the math "so that you can do it by hand with a pencil".
- You can implement an integer cube root in your language, format the message block you want to forge, leaving sufficient trailing zeros at the end to fill with garbage, then take the cube-root of that block.

Forge a 1024-bit RSA signature for the string "hi mom". Make sure your implementation actually accepts the signature!

</div>

Here's [Hal Finney's write-up](https://mailarchive.ietf.org/arch/msg/openpgp/5rnE9ZRN1AokBVj3VqblGlP63QE/).  It doesn't look like Bleichenbacher actually formally published the attack in a peer-reviewed paper -- it was presented at a Rump session of a conference -- Hal Finney did this write-up as a summary of that presentation.

In [2]:
key_size = 1024
[e,d,n] = cp.genRSA_keypair(key_size)

message = b'hi mom'
m = int(message.hex(), 16)

---
First, demo operation of signing a message and verifying a sig:
    

In [3]:
def gen_sig(message, d, n):

    msg_hash = sha1.SHA1(message).finish()
    
    # This isn't correct, but will work for this exercise
    ASN1_data = b'HASH=SHA1'
    
    num_FFs = n.bit_length()//8 - 3 - len(ASN1_data) - len(msg_hash)
    sig_to_encrypt = b'\x00\x01' + b'\xff'*num_FFs + b'\x00' + ASN1_data + msg_hash
    sig = pow(int(sig_to_encrypt.hex(), 16), d, n)
    
    return(sig)

In [4]:
sig = gen_sig(message, d, n)

In [5]:
def check_sha1_sig(message, sig, n):
    
    hexstr = hex(pow(sig, 3, n))[2:]
    # the hex() function sometimes gives us an odd # of hex characters.  
    # Add a leading 0 if necessary
    if len(hexstr) % 2 == 1:
        hexstr = '0' + hexstr
    p = bytes.fromhex(hexstr)
    hash_idx = p.find(b'HASH=SHA1') + 9
    provided_hash = p[hash_idx:hash_idx+20]
    calculated_hash = sha1.SHA1(message).finish()
                 
    if provided_hash == calculated_hash:
        return True
    else:
        return False
    

In [6]:
check_sha1_sig(message, sig, n)

True

---

<div class="alert alert-block alert-info">
You can implement an integer cube root in your language, format the message block you want to forge, leaving sufficient trailing zeros at the end to fill with garbage, then take the cube-root of that block.

Forge a 1024-bit RSA signature for the string "hi mom". Make sure your implementation actually accepts the signature!

</div>

In [7]:
# Not implementing proper ASN1 data.  Just faking it to match the characteristics above.

ASN1_data = b'HASH=SHA1'
forged_sig = b'\x00\x01\xff\x00' + ASN1_data + sha1.SHA1(message).finish() 
N_Garbage_Bytes = 1024//8 - len(forged_sig)
forged_sig = forged_sig + random.Random.get_random_bytes(N_Garbage_Bytes)
forge_sig_done = cp.root(3, int(forged_sig.hex(), 16))

---
Now check to see if our forged signature passes authentication:

In [8]:
if check_sha1_sig(message, forge_sig_done, n):
    print('\n*********************\nAuthentication passed\n*********************')
else:
    print('\n*********************\nAuthentication failed\n*********************')


*********************
Authentication passed
*********************


---

#### **TODO**:  Implement using approach from Hal Finney's write-up.

Summary of attack from Hal Finney's write-up:

- Let `D` represent the numeric value of the `00` byte, the `ASN.1 data`, and the hash, considered as a byte string.  In the case of SHA-1 this will be `36` bytes or `288` bits long.  
- Define `N` as `2^288-D`. We will assume that `N` is a multiple of `3`, which can easily be arranged by slightly tweaking the message if neccessary.

Bleichenbacher uses an example of a 3072 bit key, and he will position
the hash 2072 bits over from the right.  This improperly padded version
can be expressed numerically as 2^3057 - 2^2360 + D * 2^2072 + garbage.
This is equivalent to 2^3057 - N*2^2072 + garbage.  Then, it turns out
that a cube root of this is simply 2^1019 - (N * 2^34 / 3), and that is
a value which broken implementations accept as an RSA signature.

You can cube this mentally, remembering that the cube of (A-B) is A^3 -
3(A^2)B + 3A(B^2) - B^3.  Applying that rule gives 2^3057 - N*2^2072
+ (N^2 * 2^1087 / 3) - (N^3 * 2^102 / 27), and this fits the pattern
above of 2^3057 - N*2^2072 + garbage.  This is what Daniel means when
he says that this attack is simple enough that it could be carried out
by pencil and paper (except for the hash calculation itself).

[Back to Index](CryptoPalsWalkthroughs_Cobb.ipynb)